In [ ]:
pip install pandas openpyxl

In [ ]:
from tkinter import filedialog
from tkinter import *
root = Tk()
root.withdraw()

import pandas as pd

import os
import openpyxl
import time

import statistics

import matplotlib.pyplot as plt


In [ ]:
file_path_marks = filedialog.askopenfilename(title="Select file listing the supervisor, 2nd reader and final marks)")
print(file_path_marks)

In [ ]:
# Read the 'Students' sheet from the 'data' file into a pandas DataFrame
sheet_name = 'Data'
try:
    df = pd.read_excel(file_path_marks, sheet_name=sheet_name)
    print(f"{sheet_name} data loaded successfully!")
except Exception as e:
    print(f"An error occurred while loading {sheet_name} data: {e}")

In [ ]:
# Convert marks to numeric (assuming non-numeric values may cause issues)
df['Mark Supervisor'] = pd.to_numeric(df['Mark Supervisor'], errors='coerce')
df['Mark 2nd Reader'] = pd.to_numeric(df['Mark 2nd Reader'], errors='coerce')
df['Mark Final'] = pd.to_numeric(df['Mark Final'], errors='coerce')
df['Mark Difference'] = pd.to_numeric(df['Mark Difference'], errors='coerce')

# Collect unique academics from both Supervisor and 2nd Reader
academics = pd.concat([df['Supervisor'], df['2nd Reader']]).unique()

# Preparing the result DataFrame
results_df = pd.DataFrame({
    'Academic': academics,
    'Count': 0,
    'Average Mark': 0,
    'Average Mark Diff.': 0,
    'Average ABS-Diff.': 0,
    'Average ABS-Diff. Diff.': 0

}).set_index('Academic')

#print(df)
filtered_df = df[df['Mark Final'] != 0]
#print(filtered_df)



# Calculate average marks and differences
all_marks = []
all_diffs = []
for academic in academics:
    # supervisor marks
    is_supervisor = filtered_df['Supervisor'] == academic
    #print(type(is_supervisor))
    marks = filtered_df.loc[is_supervisor, 'Mark Supervisor'].values.tolist()
    diffs = filtered_df.loc[is_supervisor, 'Mark Difference'].values.tolist()

    # 2nd reader marks
    is_reader = filtered_df['2nd Reader'] == academic   
    marks += filtered_df.loc[is_reader, 'Mark 2nd Reader'].values.tolist()
    diffs += filtered_df.loc[is_reader, 'Mark Difference'].values.tolist()
    marks_average = statistics.mean(marks)
    marks_count = len(marks)
    diffs_average = statistics.mean(diffs)

    #print(academic, marks, marks_average, diffs, diffs_average)

    results_df.loc[academic, 'Average Mark'] = marks_average
    results_df.loc[academic, 'Count'] = marks_count
    results_df.loc[academic, 'Average ABS-Diff.'] = diffs_average 

    all_marks += marks
    all_diffs += diffs

# Resetting index to turn Academic back to a column (if you want)
results_df.reset_index(inplace=True)
#print(results_df)

all_marks_average = statistics.mean(all_marks)
all_diffs_average = statistics.mean(all_diffs)
#print(all_marks_average, all_diffs_average)


for index, academic_result in results_df.iterrows():

    results_df.loc[index, 'Average Mark Diff.'] = academic_result['Average Mark'] - all_marks_average
    results_df.loc[index, 'Average ABS-Diff. Diff.'] = academic_result['Average ABS-Diff.'] - all_diffs_average


print(results_df)

In [ ]:
plt.figure(figsize=(8, 5))  # Customizes the size of the plot (width, height in inches)
plt.scatter(results_df['Average Mark'], results_df['Average ABS-Diff.'])  # Scatter plot of 'Mark' vs 'Mark 2'
# Annotating each point
for i in range(len(results_df)):
    plt.annotate(results_df['Academic'][i],  # Text to display
                 (results_df['Average Mark'][i], results_df['Average ABS-Diff.'][i]),  # Point to annotate
                 textcoords="offset points",  # How to position the text
                 xytext=(0,10),  # Distance from text to points (x,y)
                 ha='center')  # Horizontal alignment can be left, right or center

plt.title('Scatter plot of Average Mark vs. Average ABS-Diff.')  # Adds a title
plt.xlabel('Average Mark Diff.')  # Label for the horizontal axis
plt.ylabel('Average ABS-Diff. Diff.')  # Label for the vertical axis
plt.grid(True)  # Enable grid for easier visualization
plt.show()  # Displays the plot

In [ ]:
folder_results = os.path.dirname(os.path.abspath(file_path_marks))

timestr = time.strftime("%Y%m%d-%H%M%S")
file_internal_results = "04_Output_" + timestr + "_Internal.xlsx"

filepath_internal_results = os.path.join(folder_results,file_internal_results)

wb = openpyxl.Workbook()
wb.save(filepath_internal_results)


# Use ExcelWriter to write to the specified filename
with pd.ExcelWriter(filepath_internal_results, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    results_df.to_excel(writer, sheet_name='Mark Statistics', index=False)

print(f"Sheet 'Final Allocations' has been successfully written to {filepath_internal_results}")
